# Library

In [1]:
import numpy as np
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import torch
import argparse
import time
import pickle

#from src.self_awareness.networks import utils
#from src.self_awareness.learning.tf_cnn_auxiliary_gp import Model
from torch.distributions import Normal
import matplotlib.pyplot as plt
import random

import roslib
import rospy
import tf as tf_ros
from nav_msgs.msg import Odometry, Path
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
from geometry_msgs.msg import PoseStamped, PoseArray, Pose
import math
import cv2
import copy

import sys
sys.path.append('..')

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


## Check GPU

In [2]:
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(1))

TITAN Xp


## Set torch default parameters

In [3]:
torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=4,sci_mode=False)
torch.backends.cudnn.benchmark = True

## Init ROS

In [4]:
rospy.init_node('global_localization_tf_broadcaster_cnn_gp')

# Set Arguments

In [5]:
import argparse
import sys
import os
import time
import pickle

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=1, help='size of mini batch')
parser.add_argument('--target_image_size', default=[300, 300], nargs=2, type=int, help='Input images will be resized to this for data argumentation.')

parser.add_argument('--model_dir', type=str, default='/notebooks/global_localization/gp_net_torch', help='model directory')

parser.add_argument('--test_dataset', type=str, default=[# '/notebooks/michigan_nn_data/2012_01_08',
                                                         # '/notebooks/michigan_nn_data/2012_01_15',
                                                         # '/notebooks/michigan_nn_data/2012_01_22',
                                                         # '/notebooks/michigan_nn_data/2012_02_02',
                                                         # '/notebooks/michigan_nn_data/2012_02_04',
                                                         # '/notebooks/michigan_nn_data/2012_02_05',
                                                         '/notebooks/michigan_nn_data/2012_02_12',
                                                         # '/notebooks/michigan_nn_data/2012_03_31',
                                                         '/notebooks/michigan_nn_data/2012_04_29',
                                                         '/notebooks/michigan_nn_data/2012_05_11',
                                                         '/notebooks/michigan_nn_data/2012_06_15',
                                                         '/notebooks/michigan_nn_data/2012_08_04',
                                                         # '/notebooks/michigan_nn_data/2012_09_28'])
                                                         '/notebooks/michigan_nn_data/2012_10_28',
                                                         '/notebooks/michigan_nn_data/2012_11_16',
                                                         '/notebooks/michigan_nn_data/2012_12_01'
                                                        ] )

parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/test'])
#parser.add_argument('--map_dataset', type=str, default='/home/kevin/data/michigan_gt/training')
sys.argv = ['']
args = parser.parse_args()

# Load Dataset

In [6]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import tf.transformations as tf_tran
from tqdm import tqdm
#from PIL import Image
import numpy as np
import random

#import gpflow.multioutput.kernels as mk
import gpytorch

import torch.nn as nn
import torch.optim as optim
from torchlib import resnet, vggnet
from torchlib.utils import LocalizationDataset
import time

transform = transforms.Compose([transforms.ToTensor()])
dataset = LocalizationDataset(dataset_dirs = args.test_dataset, \
                              image_size = args.target_image_size, \
                              transform = transform,
                              get_pair = False, mode='evaluate')
#[args.norm_mean, args.norm_std] = [torch.tensor(x) for x in dataset.get_norm()]
#args.norm_mean = torch.Tensor([-114.69805908,  405.21035767,   -8.72568321])
#args.norm_std = torch.Tensor([119.66057587, 176.14263916,   4.68300915])
[args.norm_mean, args.norm_std] = torch.load('/notebooks/global_localization/norm_mean_std.pt')

dataloader = DataLoader(dataset, batch_size=args.batch_size, \
                        shuffle=False, num_workers=0, \
                        drop_last=False, pin_memory=True)

 11%|█▏        | 1638/14301 [00:02<00:17, 744.63it/s]


TypeError: int() argument must be a string, a bytes-like object or a number, not 'PngImageFile'

# Define Model

In [ ]:
def denormalize_navie(normed_target, norm_mean, norm_std):
    target_trans_unscaled = normed_target * norm_std
    target_trans_uncentered = target_trans_unscaled + norm_mean
    
    return target_trans_uncentered

def denormalize(normed_target, norm_mean, norm_std):
    normed_target_trans, normed_target_rot = torch.split(normed_target, [3,4], dim=1)
    target_trans_unscaled = normed_target_trans * norm_std
    target_trans_uncentered = target_trans_unscaled + norm_mean
    target = torch.cat([target_trans_uncentered, normed_target_rot],dim=1)
    return target

def normalize(target, norm_mean, norm_std):
    target_trans = target[:,:3]
    target_trans = torch.div(torch.sub(target_trans,norm_mean),norm_std)
    target_normed = torch.cat([target_trans,target[:,3:]],dim=1)
    return target_normed 

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = resnet.resnet50(pretrained=True)
        self.global_context = vggnet.vggnet(input_channel=2048,opt="context")
        self.global_regressor = vggnet.vggnet(opt="regressor")
        
    def forward(self,input_data):
        dense_feat = self.resnet(input_data)
        global_context_feat = self.global_context(dense_feat)
        global_output, trans_feat, rot_feat = self.global_regressor(global_context_feat)
        return global_output, trans_feat, rot_feat
    
class MultitaskGPModel(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        # We have to mark the CholeskyVariationalDistribution as batch
        # so that we learn a variational distribution for each task
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            inducing_points.size(-2), batch_shape=torch.Size([3])
        )

        # We have to wrap the VariationalStrategy in a MultitaskVariationalStrategy
        # so that the output will be a MultitaskMultivariateNormal rather than a batch output
        variational_strategy = gpytorch.variational.MultitaskVariationalStrategy(
            gpytorch.variational.VariationalStrategy(
                self, inducing_points, variational_distribution, learn_inducing_locations=True
            ), num_tasks=3
        )

        super().__init__(variational_strategy)

        # The mean and covariance modules should be marked as batch
        # so we learn a different set of hyperparameters
        #self.net = Model()
        #self.net.load_state_dict(torch.load(os.path.join(args.model_dir,'model-23-96000.pth')))
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([1]))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size([1])),
            batch_shape=torch.Size([1])
        )

    def forward(self, x):
        # The forward function should be written as if we were dealing with each output
        # dimension in batch
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
class GPModel(gpytorch.Module):
    def __init__(self, inducing_points):
        super(GPModel, self).__init__()
        self.net = Model()
        #self.net.load_state_dict(torch.load(os.path.join('/notebooks/global_localization/dual_resnet_torch','model-23-96000.pth')))
        self.gp = MultitaskGPModel(inducing_points)
        self.likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=3)

    def forward(self, x):
        global_output, trans_feat, _ = self.net(x)
        _, rot_pred = torch.split(global_output, [3, 4], dim=1)
        output = self.gp(trans_feat)
        
        return output,rot_pred

In [ ]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(device)

#model = GPModel(torch.zeros(3, args.batch_size, 128)).to(device)
model = GPModel(torch.zeros(3, 300, 128)).to(device)
#model.load_state_dict(torch.load(os.path.join(args.model_dir,'pretrained.pth')))
model.load_state_dict(torch.load(os.path.join(args.model_dir,'model-14-6000.pth')))

# Disable net
for param in model.parameters():
    param.requires_grad = False

In [ ]:
args.norm_mean = args.norm_mean.to(device)
args.norm_std = args.norm_std.to(device)

In [ ]:
trans_errors = []
rot_errors = []
uncertainties = []
pose_map = []

total_trans_error = 0.
total_rot_error = 0.

count = 0.

is_save_map = False
is_read_map = False

trans_preds = []
trans_gts = []

rot_preds = []
rot_gts = []

pred_uncertainties = []

pred_time = []

In [ ]:
br = tf_ros.TransformBroadcaster()

GT_POSE_TOPIC = '/gt_pose'
BIRDVIEW_TOPIC_PUB = '/bird_view'
MAP_TOPIC_PUB = '/pose_map'
PARTICLES_PUB = '/particles'
NN_LOCALIZASION_PUB = '/nn_pose'
gt_pose_pub = rospy.Publisher(GT_POSE_TOPIC, Odometry, queue_size=1)
bird_view_pub = rospy.Publisher(BIRDVIEW_TOPIC_PUB, Image, queue_size=1)
map_pub = rospy.Publisher(MAP_TOPIC_PUB, Path, queue_size=1)
particles_pub = rospy.Publisher(PARTICLES_PUB, PoseArray, queue_size=1)
nn_pose_pub = rospy.Publisher(NN_LOCALIZASION_PUB, Odometry, queue_size=1)

In [ ]:
model.eval()
model.gp.eval()
model.likelihood.eval()

In [ ]:
def get_output(output,rot_pred,model,i=0):
    c_mean, c_var = output.mean,output.variance
    y_mean, y_var = model.likelihood(output).mean, model.likelihood(output).variance
    
    dist = Normal(c_mean, c_var.mul(args.norm_std))
    samples = dist.sample([100]).view(100,3)

    distribution_mean = c_mean
    distribution_cov = c_var.mul(args.norm_std)
    trans_prediction = denormalize_navie(y_mean,args.norm_mean,args.norm_std)
    rot_prediction = rot_pred
    #samples = denormalize_navie(samples,args.norm_mean,args.norm_std)
    return trans_prediction, rot_prediction, distribution_mean, distribution_cov, samples

for b, data in enumerate(dataloader, 0):
    start = time.time()
    x,y = data.values()
    x,y = x.to(device),y.to(device)
    #y = normalize(y,args.norm_mean, args.norm_std)
    
    # Get single data & transform data type
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        output,rot_pred = model(x)
    rot_pred = rot_pred.cpu()
    trans_pred, rot_pred, trans_mean, trans_cov, samples = get_output(output,rot_pred,model)
    trans_pred = np.asarray(trans_pred.cpu())
    rot_pred = np.asarray(rot_pred.cpu())
    trans_mean = np.asarray(trans_mean.cpu())
    trans_cov = np.asarray(trans_cov.cpu())
    samples = np.asarray(samples.cpu())
    
    end = time.time()
    pred_time.append(end-start)
    
    particles = PoseArray()
    particles.header.stamp = rospy.Time.now()
    particles.header.frame_id = 'world'
    for s in samples:
        pose = Pose()
        [pose.position.x, pose.position.y, pose.position.z] = s
        [pose.orientation.x, pose.orientation.y, pose.orientation.z, pose.orientation.w] = rot_pred[0]
        particles.poses.append(pose)
    particles_pub.publish(particles)
    
    y = np.asarray(y.cpu())
    trans_gt = y[:, :3]
    rot_gt = y[:, -4:]
    [px_pred, py_pred, pz_pred] = trans_pred[0]
    [qx_pred, qy_pred, qz_pred, qw_pred] = rot_pred[0]
    
    br.sendTransform((px_pred, py_pred, pz_pred),
                     (qx_pred, qy_pred, qz_pred, qw_pred), rospy.Time.now(),
                     "estimation", "world")
    
    [px_gt, py_gt, pz_gt] = trans_gt[0]
    [qx_gt, qy_gt, qz_gt, qw_gt] = rot_gt[0]
    
    br.sendTransform((px_gt, py_gt, pz_gt),
                     (qx_gt, qy_gt, qz_gt, qw_gt),
                     rospy.Time.now(), "gt", "world")

    timestamp = rospy.Time.now()
    
    nn_pose_msg = Odometry()
    nn_pose_msg.header.frame_id = 'world'
    nn_pose_msg.header.stamp = timestamp
    nn_pose_msg.child_frame_id = 'base_link'
    nn_pose_msg.pose.pose.position.x = px_pred
    nn_pose_msg.pose.pose.position.y = py_pred
    nn_pose_msg.pose.pose.position.z = pz_pred
    [nn_pose_msg.pose.pose.orientation.x, nn_pose_msg.pose.pose.orientation.y, nn_pose_msg.pose.pose.orientation.z, nn_pose_msg.pose.pose.orientation.w] = [qx_pred, qy_pred, qz_pred, qw_pred]
    
    conv = np.zeros((6,6), dtype=np.float32)
    [conv[0][0], conv[1][1], conv[2][2]] = trans_cov[0]
    nn_pose_msg.pose.covariance = conv.flatten().tolist()
    nn_pose_pub.publish(nn_pose_msg)
    
    bridge = CvBridge()

    bird_view_img_msg = bridge.cv2_to_imgmsg(np.asarray(x[0].cpu(), dtype=np.float32), encoding="passthrough")
    stamp_now = rospy.Time.now()
    bird_view_img_msg.header.stamp = stamp_now

    bird_view_pub.publish(bird_view_img_msg)

    rospy.sleep(.0)
    cv2.waitKey(0)

    count += 1
    
    trans_preds.append(trans_pred[0])
    rot_preds.append(rot_pred[0])
    trans_gts.append(trans_gt[0])
    rot_gts.append(rot_gt[0])
    
    trans_error = np.sum((trans_pred[0] - trans_gt[0])**2)**0.5
    rot_error_1 = np.arccos(np.dot(rot_pred[0], rot_gt[0])) / math.pi*180
    rot_error_2 = np.arccos(np.dot(rot_pred[0], -rot_gt[0])) / math.pi * 180
    rot_error = min(rot_error_1, rot_error_2)
    
    trans_errors.append(trans_error)
    rot_errors.append(rot_error)
    uncertainties.append(np.mean(np.sum(trans_cov[0]**2)**0.5) * 1000)
    pred_uncertainties.append(trans_cov[0])
    
    total_trans_error += trans_error
    total_rot_error += rot_error
    
    display = 50

    if b % display == 0 and b > 0:
        print(
            "{}/{}, translation error = {:.3f}, rotation error = {:.3f}, time/batch = {:.3f}"
            .format(
             b,
            len(dataloader),
            total_trans_error / count,
            total_rot_error / count,
            end - start))

print("pred time", np.mean(np.array(pred_time)))
print("time std", np.std(np.array(pred_time)))

In [ ]:
import scipy.io as sio

sio.savemat('results.mat', {'trans_pred': np.array(trans_preds), 'trans_gt': np.array(trans_gts), 'uncertainty': np.array(pred_uncertainties)})

if len(pose_map):
    np.savetxt(os.path.join(args.map_dataset, 'map.txt'), np.asarray(pose_map, dtype=np.float32))
    print("map is saved!")

plt.hist(trans_errors, bins='auto')
plt.title("Translation errors")
plt.xlabel("translational error in meters")
plt.ylabel("number of frames")
plt.savefig('terror.png', bbox_inches='tight')

plt.hist(rot_errors, bins='auto')
plt.title("Rotation errors")
plt.xlabel("rotational error in degree")
plt.ylabel("number of frames")
plt.savefig('rerror.png', bbox_inches='tight')

median_trans_errors = np.median(trans_errors)
median_rot_errors = np.median(rot_errors)
mean_trans_errors = np.mean(trans_errors)
mean_rot_errors = np.mean(rot_errors)

print("median translation error = {:.3f}".format(median_trans_errors))
print("median rotation error = {:.3f}".format(median_rot_errors))
print("mean translation error = {:.3f}".format(mean_trans_errors))
print("mean rotation error = {:.3f}".format(mean_rot_errors))   

In [ ]:
print('Model parameters:', sum(param.numel() for param in model.parameters()))

In [ ]:
t = [14301,7008,12852,9567,13580,14835,7114,12683]
for i in range(len(t)):
    if i >0:
        t[i] += t[i-1]
t

In [ ]:
trans_errors_month = list()
trans_errors_month.append(trans_errors[:t[0]])
trans_errors_month.append(trans_errors[t[0]:t[1]])
trans_errors_month.append(trans_errors[t[1]:t[2]])
trans_errors_month.append(trans_errors[t[2]:t[3]])
trans_errors_month.append(trans_errors[t[3]:t[4]])
trans_errors_month.append(trans_errors[t[4]:t[5]])
trans_errors_month.append(trans_errors[t[5]:t[6]])
trans_errors_month.append(trans_errors[t[6]:])

rot_errors_month = list()
rot_errors_month.append(rot_errors[:t[0]])
rot_errors_month.append(rot_errors[t[0]:t[1]])
rot_errors_month.append(rot_errors[t[1]:t[2]])
rot_errors_month.append(rot_errors[t[2]:t[3]])
rot_errors_month.append(rot_errors[t[3]:t[4]])
rot_errors_month.append(rot_errors[t[4]:t[5]])
rot_errors_month.append(rot_errors[t[5]:t[6]])
rot_errors_month.append(rot_errors[t[6]:])

In [ ]:
for trans_errors_i in trans_errors_month:
    print("median translation error = {:.3f}".format(np.median(trans_errors_i)))

In [ ]:
for rot_errors_i in rot_errors_month:
    print("median rotation error = {:.3f}".format(np.median(rot_errors_i)))

In [ ]:
for trans_errors_i in trans_errors_month:
    print("mean translation error = {:.3f}".format(np.mean(trans_errors_i)))

In [ ]:
for rot_errors_i in rot_errors_month:
    print("mean rotation error = {:.3f}".format(np.mean(rot_errors_i)))